In [38]:
from tasrif.processing_pipeline.custom import AggregateOperator, AddDurationOperator
%load_ext autoreload
%autoreload 2

from tasrif.processing_pipeline import ProcessingPipeline
from tasrif.processing_pipeline.pandas import ConvertToDatetimeOperator, SortOperator
from tasrif.processing_pipeline.custom import CreateFeatureOperator

from tasrif.data_readers.sleep_health import SleepQualityCheckerDataset
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# Full SleepQualityCheckerDataset
sqc = SleepQualityCheckerDataset(shc_folder="../../data/sleephealth/", pipeline=None)
df = sqc.raw_df.copy()
print("Shape:", df.shape)
df.head()

Shape: (42567, 3)


,participantId,sq_score,timestamp
0,b4ebf7dd-4e30-4f7b-8ee8-5493a19c8c9f,4,2016-09-26T21:18:24-05:00
1,c3993552-69cb-45e4-b18a-5e6eecefb825,4,2016-03-07T09:16:09-05:00
2,78f60bd3-34f3-489e-a352-f9df564641c3,4,2016-03-05T17:21:46-05:00
3,9da1a89a-2145-4cca-b356-7b58aa7be8b0,4,2016-09-26T21:41:55-05:00
4,4aad9dbe-dd9e-4832-a198-3bd563457124,4,2016-03-03T10:52:42-08:00


In [49]:
sqc.participant_count()

4566

In [40]:

df0 = pd.DataFrame([['Doha', 25, 30], ['Doha', 17, 50], ['Dubai', 20, 40], ['Dubai', 21, 42]],
                    columns=['city', 'min_temp', 'max_temp'])

operator = AggregateOperator(groupby_feature_names ="city",
                             aggregation_definition= {"min_temp": ["mean", "std"]})
df0 = operator.process(df0)
df0

[    city  min_temp_mean  min_temp_std
 0   Doha           21.0      5.656854
 1  Dubai           20.5      0.707107]

In [47]:
# Full SleepQualityCheckerDataset
pipeline = ProcessingPipeline([
    SortOperator(by=["participantId", "timestamp"]),
    AggregateOperator(groupby_feature_names="participantId",
                      aggregation_definition= {
                          "sq_score": ["count", "mean", "std", "min", "max", "first", "last"],
                          "timestamp": ["first", "last"]
                      }),
    ConvertToDatetimeOperator(feature_names=["timestamp_last", "timestamp_first"], format="%Y-%m-%dT%H:%M:%S%z", utc=True),
    CreateFeatureOperator(feature_name="delta_first_last_timestamp",
                          feature_creator=lambda row: row['timestamp_last'] - row['timestamp_first'])
    ])

sqc = SleepQualityCheckerDataset(shc_folder="../../data/sleephealth/", pipeline=pipeline)
df = sqc.processed_dataframe()
print("Shape:", df.shape)
df.head()


Shape: (4566, 11)


0       0 days 09:36:27
1      23 days 23:32:08
2       0 days 00:00:00
3      10 days 22:14:50
4       7 days 17:36:07
             ...       
4561    0 days 10:39:01
4562   24 days 08:26:08
4563   11 days 14:17:59
4564    0 days 00:00:00
4565    9 days 22:20:31
Name: delta_first_last_timestamp, Length: 4566, dtype: timedelta64[ns]

In [50]:
sqc.participant_count()

4566